In [1]:
!pip install numpy==1.21.5
!pip install nptyping==1.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ornstein_uhlenbeck as ou
import time
import math

In [3]:
# Global Variables

ratedCap = 8 #MW
initElecPrice = 45 #£/mwh 
capFactor = 0.3 #Output is 30% of RC
availability = 0.95 #Assume turbines are on 95% of the time
numberOfTurbines=100

inflationRate = 1.05
riskAdjRate = 0.07 #risk-adjusted discount rate
riskFreeRate = 0.03

assetLife = 20
gearBoxLife = 10

impCostGearbox = 200 # millions

estimate_num = 120   # The estimated number of data


In [4]:
# read data
df = pd.read_csv('./data/electricity-prices-day-a.csv')
df.columns = ['date','price']
df = df.head(estimate_num)

In [5]:
# Legacy Function Definitions

def netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield


def compute_price_parameters(df):
    """
    this function compute parameters of OU process
    :param df: price dataframe
    :return mu: ou process mu
    :return theta: ou process theta
    :return sigma_sq: ou process sigma_sq 
    """
    date_list = df['date'].to_list()
    x_data = np.array(df['price'].to_list())
    #stamp_list  = [int(time.mktime(time.strptime(i, "%Y-%m-%d %H:%M:%S"))) for i in date_list]
    stamp_list = [i for i in range(len(x_data))]
    estimator = ou.OrnsteinUhlenbeckEstimator([(np.array(stamp_list), x_data)], n_it=5)
    mu = estimator.mu
    theta = estimator.eta
    sigma_sq = estimator.sigma_sq()
    x_t = x_data[-1]
    return mu,theta,sigma_sq,x_t


#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def cashFlowCalculation(*args):
    discountedCashFlows = np.zeros(assetLife)

    mu,theta,sigma_sq,x_t = compute_price_parameters(df)

    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        x_t_plus_1 = x_t*math.exp(-theta)+mu*(1-math.exp(-theta))+sigma_sq/(2*theta)*(1-math.exp(-theta*2))
        discountedCashFlows[i] = pow(10,-6)* 0.7 * x_t_plus_1 * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        x_t = x_t_plus_1
        
    return discountedCashFlows


#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def baseCashFlowCalculation(*args):
    discountedBaseCashFlows = np.zeros(assetLife)

    
    
    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        discountedBaseCashFlows[i] = pow(10,-6)* 0.7 * initElecPrice * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        
    return discountedBaseCashFlows

In [6]:
# Simulation Environment Functions

def TurbineDeathSimulation(scale, loc):
    """Takes the baseCashFlows and simulates how turbine failures would change overall project npv"""
    
    discountedBaseCashFlows = baseCashFlowCalculation()
    x= np.floor(np.random.normal(loc = loc , scale = scale , size = numberOfTurbines)).astype('int')
    unique, counts = np.unique(x, return_counts=True)
    y = np.asarray((unique, 1 - np.cumsum(counts)/numberOfTurbines)).T

    if np.any(y<0):
        y = np.asarray([ [ i[0]-min(y[:,0]) , i[1] ] for i in y ])
        
        if np.any(y>assetLife-1):
            y = y[y[:,0] < assetLife]
            y[-1][1] = 0

    elif np.any(y>assetLife-1):
        y = np.asarray([ [ i[0] - ( max(y[:,0]) - (assetLife-1) ) , i[1] ] for i in y ]) 

        if np.any(y<0):
            startValue = y[0][1]
            y = y[y[:,0] > -1]
            y[0][1] = startValue
            
            
    
    probArray = np.ones(assetLife)
    for count,i in enumerate(y):
        probArray[i[0].astype('int')] = i[1]
    probArray = [0 if count>y[-1][0] else i for count,i in enumerate(probArray)]    
    
    output = discountedBaseCashFlows * probArray
    
    return output[output > 0]
    

    
    
def SigmaCalculator(testFunction,arguments):
    """Takes the price fluctuated cash flows and calculates the mean of the standard deviation of the
    logrithemic returns ,from it"""
    
    sigma_array = []
    for _ in range(0,10000):
        
        discountedCashFlows = testFunction(arguments)
        logrithmicReturns = np.zeros(len(discountedCashFlows) - 1)
        
        for g in enumerate(discountedCashFlows):
            if g[0]<len(discountedCashFlows) - 1:
                logrithmicReturns[g[0]] = np.log( discountedCashFlows[g[0]+1]/discountedCashFlows[g[0]] )

        sigma_array.append(np.std(logrithmicReturns))

    sigma = np.mean(sigma_array)
    return sigma

In [12]:
sigma = SigmaCalculator(cashFlowCalculation,())
print('sigma = ', sigma)

sigma =  0.08542013087237595
